In [1]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_parquet('./data/catB_train.parquet')

In [2]:
# Add new feature Age
df = df[df['cltdob_fix']!='None']
df['cltdob_fix'] = pd.to_datetime(df.iloc[:, 6], format ='mixed')
df['age'] = 2024-df['cltdob_fix'].dt.year

In [3]:
# Target Column
df["f_purchase_lh"] = df["f_purchase_lh"].fillna(0)
y = df["f_purchase_lh"]

# All features 
X = df.drop(columns=['f_purchase_lh'])

In [4]:
# Split numerical and non-numerical columns
numeric_cols = X.select_dtypes(include=["int32", "int64", "float64"]).columns
X_numeric = X[numeric_cols]

In [6]:
# Remove Low-Variance Numerical Variables
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(0.05))
sel.fit(X_numeric)
X_numeric = X_numeric[numeric_cols[sel.get_support()]]

In [7]:
# Fill null values in numeric columns with the median value
X_numeric = X_numeric.apply(lambda x: x.fillna(x.median()))

In [8]:
# Merge with selected non_categorical values
temp = pd.get_dummies(X[['cltsex_fix', 'stat_flag']], dtype=float)
X = pd.concat([X_numeric, temp, df['age']], axis=1)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Mapping for replacement for categorical data (not hot encoding)
mapping = {
    None: -1,
    'E.BELOW30K': 0,
    'D.30K-60K': 1,
    'C.60K-100K': 2,
    'B.100K-200K': 3,
    'A.ABOVE200K': 4,
}

# Replace values based on the mapping
df['annual_income_est'] = df['annual_income_est'].replace(mapping)